In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

In [0]:
# Read recipe inputs
weather_downtimes_12m = dataiku.Dataset("weather_downtimes_12m")
weather_downtimes_12m_df = weather_downtimes_12m.get_dataframe()

In [0]:
df2 = weather_downtimes_12m_df.groupby('DOWNTIME_ENTRY_ID', as_index=False)['weather_score','time_to_end_min'].agg(list)
#df2.head(10)

In [0]:
#for row in df2['time_to_end_min']:
#    print(row)

In [0]:
#for row in df2['weather_score']:
#    print(row)

In [0]:
data = []
for row in df2['weather_score']:
    if any(x<2 for x in row):
        data.append((list(map(lambda i: i<2, row)).index(True)))
    else:
        data.append(len(row)-1)
#print(data)

In [0]:
data2 = []
for row in df2['weather_score']:
    if any(x<2 for x in row):
        data2.append((list(map(lambda i: i<2, row)).index(True)))
    else:
        data2.append('NA')

In [0]:
time_list = df2.time_to_end_min.values.tolist()
#time_list

recovery_time=[time_list[i][data[i]] for i in range(len(data))]
#recovery_time

In [0]:
for i in range(len(recovery_time)):
    if data2[i]=='NA':
        recovery_time[i]='NA'
#recovery_time

In [0]:
df = pd.DataFrame({'Recovery Time (minutes)':recovery_time})
df['DOWNTIME_ENTRY_ID']=df2['DOWNTIME_ENTRY_ID']
df=df[['DOWNTIME_ENTRY_ID','Recovery Time (minutes)']]
#df.head(25)

In [0]:
#rslt_df = df[df['DOWNTIME_ENTRY_ID'] == 1643539]
#rslt_df.head()

In [0]:
weather_downtimes_12m_scored_recovery_df=weather_downtimes_12m_df.merge(df, on='DOWNTIME_ENTRY_ID', how='left')


# Write recipe outputs
weather_downtimes_12m_scored_recovery = dataiku.Dataset("weather_downtimes_12m_scored_recovery")
weather_downtimes_12m_scored_recovery.write_with_schema(weather_downtimes_12m_scored_recovery_df)